In [1]:
%cd ~
%cd ../../disk2/jl/jl/data_analyzing_tool
import numpy as np

/home/eis
/disk2/jl/jl/data_analyzing_tool


/home/eis/anaconda3/envs/jl_data_analysis/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/eis/anaconda3/envs/jl_data_analysis/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
features = []  # 전역 리스트 (hook_fn에서 사용)

def hook_fn(module, input, output):
    """
    지정된 레이어(예: conv3)의 '입력' 혹은 '출력'을 캡처하는 Hook 함수.
    여기서는 input[0]을 사용하므로, conv3 레이어의 입력 텐서를 수집합니다.
    
    - 어떤 레이어에 걸고 싶은지는 모델 구조에 따라 변경하세요.
    - 혹은 output을 보려면 output을 사용하세요.
    """
    # Hook에 들어오는 input[0]은 (batch_size, channel, height, width) 형태일 가능성이 큽니다.
    # 필요하다면 .detach() 처리 후 numpy()로 변환하세요.
    # GPU->CPU 변환 위해 .cpu() 사용, 미분 그래프에서 분리 위해 .detach() 사용
    batch_features = input[0].detach().cpu().numpy()
    features.append(batch_features)
    
def extract_per_sample_features(model, inference_instance, layer_name):
    """
    주어진 모델과 inference_instance(= 데이터셋 + forward)를 이용해
    전체 데이터셋 각 샘플별 feature를 추출하여 반환.
    """
    global features
    features = []  # 매 호출 시 초기화

    # Hook 등록 (conv3 레이어)
    # hook = model.conv3.register_forward_hook(hook_fn)
    # hook = model.conv1.register_forward_hook(hook_fn)
    
    layer = getattr(model, layer_name, None)
    hook = layer.register_forward_hook(hook_fn)

    # (아래 코드는 예시입니다. 실제론 inference_instance.examin()이
    # 전체 샘플에 대해 forward, loss/acc 계산을 수행한다고 가정)
    avg_loss, acc, all_targets, all_outputs = inference_instance.examin()

    # Hook 해제
    hook.remove()

    # features: 각 배치마다 쌓인 numpy 배열들의 리스트
    # -> shape (num_samples, C, H, W) 로 합치기
    extracted_features = np.concatenate(features, axis=0)

    # 필요에 따라 (num_samples, C*H*W) 형태로 리쉐이프
    B, C, H, W = extracted_features.shape
    extracted_features = extracted_features.reshape(B, -1)

    return extracted_features, all_targets  # (num_samples, feature_dim), (num_samples, )



In [3]:
%load_ext autoreload
%autoreload 
from data_prepare.data_prepare_manager import select_dataset

dataset_name = 'cifar-10_jg'
dataset_jg, classes_cifar10 = select_dataset(dataset_name)

dataset_name = 'cifar-10_jo'
dataset_jo, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_jp'
dataset_jp, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_jr'
dataset_jr, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_r1'
dataset_r1, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_r2'
dataset_r2, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_r3'
dataset_r3, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_r4'
dataset_r4, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_lp'
dataset_lp, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_bp'
dataset_bp, _ = select_dataset(dataset_name)

dataset_name = 'cifar-10_bs'
dataset_bs, _ = select_dataset(dataset_name)

classes = classes_cifar10
num_classes = len(classes)
print(num_classes)

All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
10


In [4]:
import os
from model_structure.testModel_wo_softmax_bigger_filters import testNN_wo_Softmax_bigger_filters
from model_structure.testModel_wo_softmax_more_filters import testNN_wo_Softmax_more_filters
from model_structure.testModel_wo_softmax_5_layer import testNN_wo_Softmax_5_layer
from cka_analysis.cka import *
from model_structure.testNN_wo_softmax import testNN_wo_Softmax
from model_structure.testModel_wo_softmax_3_layer import testNN_wo_Softmax_3_layer
# from model_structure.testModel_wo_softmax_5_layer import testNN_wo_Softmax_5_layer
from inference import inference
import torch

# rootModel-fed_avg_3_layer - SD cifar10 original | RP.pth
# rootModel-fed_avg_3_layer - SD cifar10 jitter | RP.pth
# rootModel-easy_fed_avg_3_layer - SD cifar10 rotate | RP.pth
# rootModel-fed_avg_3_layer - SD cifar10 noise | RP.pth

# rootModel-fed_prox_3_layer - SD cifar10 original | RP.pth
# rootModel-fed_prox_3_layer - SD cifar10 jitter | RP.pth
# rootModel-easy_fed_prox_3_layer - SD cifar10 rotate | RP.pth
# rootModel-fed_prox_3_layer - SD cifar10 noise | RP.pth


# rootModel-easy_fisher_client - MD Mixed | RCP.pth
# rootModel-easy_fisher_client - MD Mixed | CP 1R.pth
# rootModel-easy_fed_prox_3_layer - MD Mixed | RP.pth
# rootModel-easy_fed_avg_3_layer - MD Mixed | RP.pth
# rootModel-easy_fed_avg_3_layer - MD Mixed | CP.pth
# rootModel-easy_fed_avg_3_layer - MD Mixed | RP.pth

# rootModel-fed_prox_3_layer - MD Mixed | RP.pth
# rootModel-fed_avg_3_layer - MD Mixed | RP.pth

# rootModel-fed_avg_5_layer - MD Mixed | RP.pth
# rootModel-fed_avg_more_filters - MD Mixed | RP.pth
# rootModel-fed_avg_bigger_filters - MD Mixed | RP.pth

# rootModel-fed_prox_5_layer - MD Mixed | RP.pth
# rootModel-fed_prox_more_filters - MD Mixed | RP.pth
# rootModel-fed_prox_bigger_filters - MD Mixed | RP.pth

# rootModel-fed_avg_3_layer - preliminary | RP.pth
# rootModel-fed_avg_3_layer - preliminary IID | RP.pth

def calc_cka(baseline_dataset, comparison_dataset, baseline_str="", compare_str="", layers_list=None, baseline_path='', comparison_path=''):
    
    baseline_model = testNN_wo_Softmax_3_layer(outputClasses=num_classes)
    comparison_model = testNN_wo_Softmax_3_layer(outputClasses=num_classes)
    
    baseline_infer = inference(
        cudaId=0,             
        dataset=baseline_dataset,     
        model=baseline_model,
        pthPath=baseline_path
    )
    
    comparison_infer = inference(
        cudaId=0,              
        dataset=comparison_dataset,     
        model=comparison_model,
        pthPath=comparison_path
    )
    
    # -------------------------------------------------------
    # 2. 레이어별 Feature 추출 & CKA 계산
    # -------------------------------------------------------
    
    cka_results = {}  # 레이어별 CKA 값을 저장할 딕셔너리
    
    for layer_name in layers_list:
        print(f"\n[Layer: {layer_name}] Feature 추출 중...")
    
        # baseline 모델의 layer_name 레이어 feature 추출
        baseline_features, baseline_targets = extract_per_sample_features(
            model=baseline_model,
            inference_instance=baseline_infer,
            layer_name=layer_name
        )
    
        # comparison 모델의 layer_name 레이어 feature 추출
        comparison_features, comparison_targets = extract_per_sample_features(
            model=comparison_model,
            inference_instance=comparison_infer,
            layer_name=layer_name
        )
    
        # ---------------------------------------------------
        # 3. Feature로부터 Kernel 계산 & CKA 수행
        # ---------------------------------------------------
        K_baseline = linear_kernel(baseline_features)
        K_comparison = linear_kernel(comparison_features)
        
        cka_score = cka(K_baseline, K_comparison)
        cka_results[layer_name] = cka_score
        
        print(f"Layer [{layer_name}] - CKA Score: {cka_score:.4f}")
    
    print("\n==== 전체 레이어별 CKA 결과 ====")
    for layer_name, score in cka_results.items():
        print(f"  {layer_name} : {score:.4f}")
        
    import matplotlib.pyplot as plt
    
    # 레이어 이름과 CKA 점수를 각각 리스트로 변환
    layer_names = list(cka_results.keys())
    cka_scores = list(cka_results.values())
    
    plt.figure(figsize=(6, 4))
    plt.bar(layer_names, cka_scores, color='skyblue')
    plt.ylim(0, 1.0)  # CKA 범위가 [0,1]이므로 y축을 0~1로 제한
    plt.xlabel("Layer Name")
    plt.ylabel("CKA Score")
    plt.title(f"CKA Scores {baseline_str} : {compare_str}")
    
    # 막대 위에 수치 표시 (옵션)
    for i, score in enumerate(cka_scores):
        plt.text(i, score + 0.01, f"{score:.4f}", ha='center')
    
    dir_name = baseline_path.split('/')[-1].split('.')[0]
    if not os.path.exists(f"{dir_name}"):
            os.makedirs(f"{dir_name}")
    plt.savefig(f"{dir_name}/CKA Scores {baseline_str}-{compare_str}.png")
    # plt.show()



In [ ]:
paths = ['rootModel-fed_prox jitter mu 1e-3 | RP',
         'rootModel-fed_cka 3-1 jitter mu 1e-3 | RP',
         'rootModel-fed_avg jitter mu 1e-3 | RP']

for path_name in paths:
    baseline_path = f'./trained_pth/feature_space_guiding/{path_name}.pth'
    comparison_path = f'./trained_pth/feature_space_guiding/{path_name}.pth'
    layers_list = ["conv1", "conv2", "conv3"]
    
    dataset_list = [dataset_r1, dataset_r2, dataset_r3, dataset_r4,
                    dataset_jg, dataset_jo, dataset_jp,
                    dataset_lp, dataset_bp, dataset_bs]
    
    dataset_str_list = ["dataset_r1", "dataset_r2", "dataset_r3", "dataset_r4",
                        "dataset_jg", "dataset_jo", "dataset_jp",
                        "dataset_lp", "dataset_bp", "dataset_bs"]
    
    for baseline, baseline_name in zip(dataset_list, dataset_str_list):
        for compare, compare_name in zip(dataset_list, dataset_str_list):
            if baseline != compare: 
                calc_cka(baseline, compare, baseline_name, compare_name, layers_list, baseline_path, comparison_path)


Examin device online
cuda:0 available
Examin device online
cuda:0 available

[Layer: conv1] Feature 추출 중...
Server validation Loss: 1.7328 | accuracy:  51.1400
Server validation Loss: 4.0975 | accuracy:  20.3800
examiner offline
Layer [conv1] - CKA Score: 0.9892

[Layer: conv2] Feature 추출 중...
Server validation Loss: 1.7328 | accuracy:  51.1400
Server validation Loss: 4.0975 | accuracy:  20.3800
Layer [conv2] - CKA Score: 0.9575

[Layer: conv3] Feature 추출 중...
Server validation Loss: 1.7328 | accuracy:  51.1400
Server validation Loss: 4.0975 | accuracy:  20.3800
Layer [conv3] - CKA Score: 0.9053

==== 전체 레이어별 CKA 결과 ====
  conv1 : 0.9892
  conv2 : 0.9575
  conv3 : 0.9053
examiner offline
examiner offline
Examin device online
cuda:0 available
Examin device online
cuda:0 available

[Layer: conv1] Feature 추출 중...
Server validation Loss: 1.7328 | accuracy:  51.1400
Server validation Loss: 3.4893 | accuracy:  24.5600
Layer [conv1] - CKA Score: 0.9891

[Layer: conv2] Feature 추출 중...
Server v

/tmp/ipykernel_857161/3468197442.py:107: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(6, 4))


examiner offline
examiner offline
Examin device online
cuda:0 available
Examin device online
cuda:0 available

[Layer: conv1] Feature 추출 중...
Server validation Loss: 3.4893 | accuracy:  24.5600
Server validation Loss: 1.7325 | accuracy:  52.5300
Layer [conv1] - CKA Score: 0.9855

[Layer: conv2] Feature 추출 중...
Server validation Loss: 3.4893 | accuracy:  24.5600
Server validation Loss: 1.7325 | accuracy:  52.5300
Layer [conv2] - CKA Score: 0.9577

[Layer: conv3] Feature 추출 중...
Server validation Loss: 3.4893 | accuracy:  24.5600
Server validation Loss: 1.7325 | accuracy:  52.5300
Layer [conv3] - CKA Score: 0.9281

==== 전체 레이어별 CKA 결과 ====
  conv1 : 0.9855
  conv2 : 0.9577
  conv3 : 0.9281
examiner offline
examiner offline
Examin device online
cuda:0 available
Examin device online
cuda:0 available

[Layer: conv1] Feature 추출 중...
Server validation Loss: 3.4893 | accuracy:  24.5600
Server validation Loss: 1.6877 | accuracy:  53.7000
Layer [conv1] - CKA Score: 0.9799

[Layer: conv2] Feature